In [1]:
import json
import pandas as pd
import glob
from pandas import json_normalize

In [ ]:
NEW CONTINUING (part 1 of CR data)

note about the format of this json file <br>
FULL file one dict<br>
metadata dict<br>
lines list of dict with nested dicts<br>
outer dict in lines - name + text<br>
inner nested dict - ts timestamp information

In [2]:
#read in all json files for cr campaign 2
files = glob.glob('json_transcripts/c2/*.json' ,recursive=True)

In [3]:
#note that the order is a little odd or different from what i would have expected. last file = 99
files[:10]

['json_transcripts/c2\\cr2-1.json',
 'json_transcripts/c2\\cr2-10.json',
 'json_transcripts/c2\\cr2-100.json',
 'json_transcripts/c2\\cr2-101.json',
 'json_transcripts/c2\\cr2-102.json',
 'json_transcripts/c2\\cr2-103.json',
 'json_transcripts/c2\\cr2-104.json',
 'json_transcripts/c2\\cr2-105.json',
 'json_transcripts/c2\\cr2-106.json',
 'json_transcripts/c2\\cr2-107.json']

In [4]:
#build dict where episode number is the key and the value is the text of the json file
crdict = {}
def GetFile(filename):
    for file in files:
        results = file.split('-')  #split file path at '-'
        epnum = results[1]         #index to 1 to get episodenum.json
        epnum = epnum[:-5]         #index to get rid of '.json'
        f = open(file, 'r')        #read in files
        text = f.read()
        f.close
        crdict[epnum]=text

In [5]:
GetFile(files)
#crlist[1]

In [6]:
len(crdict)  #confirm 147 entries in dict

147

In [7]:
len(files)   #and 147 paths in imported files. 147 files for 141 episodes

147

In [8]:
ep_1 = crdict['1']    #note dict key '1' not index 1
#ep_1
#isolate one single episode to work out the formula to get the df I want

In [9]:
data = json.loads(ep_1)      #read in isolated ep1
df = json_normalize(data)
df.head()

,lines,metadata.campaign,metadata.episode,metadata.title,metadata.html_file
0,"[{'name': 'MATT', 'ts': {'h': 0, 'm': '0', 's'...",2,1,Curious Beginnings,html\cr2-1.html


In [10]:
df_test = df
#for testing and messing around with without affecting the base/first level df

In [11]:
df_test = json_normalize(df_test['lines'])  #normalize "lines" column to expand
df_test.head()     #one line from full episode per column

,0,1,2,3,4,5,6,7,8,9,...,1587,1588,1589,1590,1591,1592,1593,1594,1595,1596
0,"{'name': 'MATT', 'text': 'We've got some cool ...","{'name': 'LAURA', 'text': 'As in nnnn. ', 'ts....","{'name': 'MATT', 'text': 'Nnnn. But yeah, so t...","{'name': 'TRAVIS', 'text': '(vomiting noises) ...","{'name': 'MATT', 'text': 'So yeah. I'm super e...","{'name': 'SAM', 'text': '(laughter) ', 'ts.h':...","{'name': 'LAURA', 'text': 'Thanks, Sam! ', 'ts...","{'name': 'SAM', 'text': '(laughter) ', 'ts.h':...","{'name': 'MATT', 'text': 'It's made out of Eng...","{'name': 'SAM', 'text': 'It smells really nice...",...,"{'name': 'TRAVIS', 'text': 'I'm a ballerina. '...","{'name': 'MARISHA', 'text': 'He's a prima. ', ...","{'name': 'MATT', 'text': 'Well, guys. Happy Ne...","{'name': 'MARISHA', 'text': 'Thank you to our ...","{'name': 'SAM', 'text': 'Thank you, Matt Merce...","{'name': 'MATT', 'text': 'I'm excited. ', 'ts....","{'name': 'SAM', 'text': 'Thanks, Yev. ', 'ts.h...","{'name': 'MATT', 'text': 'Guys, have a wonderf...","{'name': 'ASHLEY', 'text': 'Yeah, me too. ', '...","{'name': 'MATT', 'text': 'Until then, have a w..."


In [12]:
df_test[0]

0    {'name': 'MATT', 'text': 'We've got some cool ...
Name: 0, dtype: object

In [13]:
ts_test_df = json_normalize(df_test[0]) 
ts_test_df    #confirmation that one final normalization gets the format I want

,name,text,ts.h,ts.m,ts.s
0,MATT,We've got some cool stuff to talk about. First...,0,0,15


In [14]:
test_long = pd.melt(df_test)    #move lines columns into rows

In [15]:
test_long.tail()

,variable,value
1592,1592,"{'name': 'MATT', 'text': 'I'm excited. ', 'ts...."
1593,1593,"{'name': 'SAM', 'text': 'Thanks, Yev. ', 'ts.h..."
1594,1594,"{'name': 'MATT', 'text': 'Guys, have a wonderf..."
1595,1595,"{'name': 'ASHLEY', 'text': 'Yeah, me too. ', '..."
1596,1596,"{'name': 'MATT', 'text': 'Until then, have a w..."


In [16]:
ts2_df = json_normalize(test_long['value'])    #normalize value column to open final nested dict

In [17]:
ts2_df.head()   #speaker name, spoken line, timestamp info

,name,text,ts.h,ts.m,ts.s
0,MATT,We've got some cool stuff to talk about. First...,0,0,15
1,LAURA,As in nnnn.,0,1,21
2,MATT,"Nnnn. But yeah, so they're going to be a long-...",0,1,24
3,TRAVIS,(vomiting noises) Would you like some?,0,2,4
4,MATT,"So yeah. I'm super excited to have that, guys....",0,2,7


In [18]:
ts2_df.shape      #confirm all lines were reached in normalization

(1597, 5)

In [19]:
#build list of normalized dataframes line by line episode by episode
emplist = []
def buildDF(feddict):         #feed dict created by GetFile function
    for file in feddict:        #for each file in dict
        info = json.loads(feddict[str(file)])     #load str of value from fed dict
        base_df = json_normalize(info)            #first level normalize
        lines_df = json_normalize(base_df['lines'])   #second level normalize
        lines_long = pd.melt(lines_df)                #melt columns into rows
        ts_df = json_normalize(lines_long['value'])   #third level normalize
        ts_df['episode']=file                         # add column to final df populated w dict key
        emplist.append(ts_df)

In [20]:
buildDF(crdict)     #feed dict into function

In [21]:
len(emplist)       #verify once more the correct number of items added to list

147

In [22]:
emplist[0]        #verify index 0 (episode 1) matches up with what we've already seen before

,name,text,ts.h,ts.m,ts.s,episode
0,MATT,We've got some cool stuff to talk about. First...,0,0,15,1
1,LAURA,As in nnnn.,0,1,21,1
2,MATT,"Nnnn. But yeah, so they're going to be a long-...",0,1,24,1
3,TRAVIS,(vomiting noises) Would you like some?,0,2,4,1
4,MATT,"So yeah. I'm super excited to have that, guys....",0,2,7,1
...,...,...,...,...,...,...
1592,MATT,I'm excited.,3,16,53,1
1593,SAM,"Thanks, Yev.",3,16,54,1
1594,MATT,"Guys, have a wonderful evening. We'll see you ...",3,16,57,1
1595,ASHLEY,"Yeah, me too.",3,17,10,1


In [23]:
cr_df = pd.concat(emplist)   #concat our list of dataframes into one big one

In [24]:
cr_df.head()     #starts with episode 1 like we expect

,name,text,ts.h,ts.m,ts.s,episode
0,MATT,We've got some cool stuff to talk about. First...,0,0,15,1
1,LAURA,As in nnnn.,0,1,21,1
2,MATT,"Nnnn. But yeah, so they're going to be a long-...",0,1,24,1
3,TRAVIS,(vomiting noises) Would you like some?,0,2,4,1
4,MATT,"So yeah. I'm super excited to have that, guys....",0,2,7,1


In [25]:
cr_df.tail()      #and ends with episode 99 which is also correct!

,name,text,ts.h,ts.m,ts.s,episode
2722,MATT,"We're going to get through this together, guys.",3,47,47,99
2723,SAM,Wash your stubbly face.,3,47,49,99
2724,TRAVIS,Gross.,3,47,51,99
2725,MATT,(cheering),3,47,52,99
2726,TRAVIS,(dramatic orchestral music),3,47,57,99


In [26]:
cr_df.shape     #that's a lot of lines

(434052, 6)

In [27]:
#there was some metadata in the initial unpacking I wanted. one of those was ep number, I also want title
metalist = []
for x in crdict:           
    fullcols = json.loads(crdict[str(x)])      
    newdf = json_normalize(fullcols)
    metalist.append(newdf)

In [28]:
meta_df = pd.concat(metalist)

In [29]:
meta_df.rename(columns={'metadata.episode':'episode', 'metadata.title':'title'}, inplace=True)
meta_df.head()

,lines,metadata.campaign,episode,title,metadata.html_file
0,"[{'name': 'MATT', 'ts': {'h': 0, 'm': '0', 's'...",2,1,Curious Beginnings,html\cr2-1.html
0,"[{'name': 'MATT', 'ts': {'h': 0, 'm': '0', 's'...",2,10,Waste and Webs,html\cr2-10.html
0,"[{'name': 'MATT', 'ts': {'h': 0, 'm': '0', 's'...",2,100,Hunted at Sea,html\cr2-100.html
0,"[{'name': 'MATT', 'ts': {'h': 0, 'm': '0', 's'...",2,101,"Mysteries, Memories, and Music",html\cr2-101.html
0,"[{'name': 'MATT', 'ts': {'h': 0, 'm': '0', 's'...",2,102,"Ghosts, Dinosaurs, and Stuff",html\cr2-102.html


I'm okay to lose the campaign column since i'm only looking at campaign 2 for this project, it would be kind of pointless to include. I'm also not worried about keeping the html file name since it is the same as the episode column, it won't come into play for my analysis. 

In [30]:
meta_df = meta_df.iloc[:, 2:4]
meta_df.head()

,episode,title
0,1,Curious Beginnings
0,10,Waste and Webs
0,100,Hunted at Sea
0,101,"Mysteries, Memories, and Music"
0,102,"Ghosts, Dinosaurs, and Stuff"


This was a decent start, but I ended up going back and amending my buildDF function to add the episode number column into the df as it was built instead, and this worked a lot better in general. I'm keeping this here though, to preserve that this was my thinking process at first. 

I ultimately decided the episode titles themselves were not entirely necessary (similar to the campaign number column). The episode number is more than enough to utilize when considering per episode stats.

In [31]:
import pickle

In [32]:
cr_df.head()

,name,text,ts.h,ts.m,ts.s,episode
0,MATT,We've got some cool stuff to talk about. First...,0,0,15,1
1,LAURA,As in nnnn.,0,1,21,1
2,MATT,"Nnnn. But yeah, so they're going to be a long-...",0,1,24,1
3,TRAVIS,(vomiting noises) Would you like some?,0,2,4,1
4,MATT,"So yeah. I'm super excited to have that, guys....",0,2,7,1


In [33]:
cr_df.tail()

,name,text,ts.h,ts.m,ts.s,episode
2722,MATT,"We're going to get through this together, guys.",3,47,47,99
2723,SAM,Wash your stubbly face.,3,47,49,99
2724,TRAVIS,Gross.,3,47,51,99
2725,MATT,(cheering),3,47,52,99
2726,TRAVIS,(dramatic orchestral music),3,47,57,99


and we have episode numbers! This is the data for Critical Role as I intend to work with it.

Now to pickle it, keep it stored in this state, and have it ready to get to work.

In [34]:
import pickle

In [35]:
#cr_df.to_pickle('pickle_jar/CritRole.pkl')